In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jul-2022/data.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.isna().sum() ## no missing values

In [ ]:
data.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

In [ ]:
figure = plt.figure(figsize = (16,8))
for i in range(29):
    feature_name = 'f_0{}'.format(i) if i < 10 else 'f_{}'.format(i) 
    plt.subplot(5, 6, i+1)
    sns.kdeplot(data[feature_name], fill = True)        
figure.tight_layout(h_pad=1.0, w_pad=0.5)
plt.suptitle('Distribution Plots', y=1.02)
plt.show()

In [ ]:
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [ ]:
data_scaled

In [ ]:
pca = decomposition.PCA(n_components = 0.99 )
data_pca = pca.fit_transform(data_scaled)
print(f'Variance explained by the principle components {pca.explained_variance_ratio_}')
print(f'Ratio of variance explained, {sum(pca.explained_variance_ratio_)}')

plt.figure(figsize = (10,10))
plt.scatter(x = data_pca[:, 0], y = data_pca[:, 1])
plt.title('First 99% principle conponents of scaled data')
plt.show()

In [ ]:
from sklearn.mixture import GaussianMixture


In [ ]:
gaussian = GaussianMixture(n_components = 7, covariance_type = 'full', init_params = 'kmeans')

In [ ]:
gaussian.fit(data_pca)

In [ ]:
y_pred = gaussian.predict(data_pca)

inertia = list()
k = range(1,10)
for iter in k:
    kmeans = KMeans(n_clusters = iter, random_state = 123)
    kmeans.fit(data_pca)
    y_means = kmeans.predict(data_pca)
    center = kmeans.cluster_centers_
    inertia.append(kmeans.inertia_)

plt.figure(figsize = (12,10))
plt.title('number of clusters')
sns.lineplot(x = k, y= inertia)
plt.ylabel('wss')
plt.xlabel('$k$')
plt.xticks(k)
plt.show()

kmeans = KMeans(n_clusters = 10)
kmeans.fit(data_pca)
y_pred = kmeans.predict(data_pca)

In [ ]:
y_means_final = pd.DataFrame(y_pred, columns = ['predicted'])

In [ ]:
y_means_final.value_counts()

In [ ]:
id = data.iloc[:, 0]

In [ ]:
id = pd.DataFrame(id, columns = ['id'])

kmeans = KMeans(n_clusters=3, random_state=123)
kmeans.fit(data_scale)
y_kmeans = kmeans.predict(data_scale)

data_kmeans = data.copy()
data_kmeans['CLUST'] = y_kmeans
sns.pairplot(data_kmeans, hue='CLUST')

In [ ]:
final = pd.concat([id, y_means_final], axis = 1)

In [ ]:
final.head()

In [ ]:
os.chdir('/kaggle/working')

In [ ]:
final.to_csv('submission_ten.csv', index = False, sep = ',')